In [1]:
import numpy as np
import pandas as pd
import pickle
# import faiss
from tqdm.notebook import tqdm

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy.stats import entropy
from scipy import spatial

/home/lliang06/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
target_researchers_profile = pd.read_csv('./data/target_researchers_profile.csv')

In [3]:
id2name = {}
for i in range(len(target_researchers_profile)):
    id2name[target_researchers_profile.loc[i]['researcher_id']] = target_researchers_profile.loc[i]['PI']

In [5]:
# lda_pubid = pd.read_csv('./data/lda_pubid.csv')
pubid = pd.read_csv('./data/lda_pubid.csv')
pubid_year = pd.read_csv('./data/pubid_year.csv')
pub2re = pd.read_parquet('./data/target_abstracts_pub2researcher.parquet')
pub_cnt = pd.read_csv('./data/pub_count.csv')
foa_abstracts = pd.read_csv('./data/energy_grants_Dimensions_wAbstract_cleanest.csv')

In [ ]:
potential_author_citing = pd.read_csv('./data/potential_author_citing.csv')
potential_author_referenced = pd.read_csv('./data/potential_author_referenced.csv')

potential_author_citing['citing'] = len(potential_author_citing) * [1]
potential_author_referenced['referenced'] = len(potential_author_referenced) * [1]

In [6]:
j = 0
all_vec = np.load(open('./data/lda/d100/embedding/d100_66_lda_part' + str(j) + '.npy', 'rb'))
for j in tqdm(range(1, 62)):
    all_vec = np.vstack([all_vec, np.load(open('./data/lda/d100/embedding/d100_66_lda_part' + str(j) + '.npy', 'rb'))])

In [29]:
with open('./data/lda/d100/FOA_d100_66.npy', 'rb') as f:
    vec_foa = np.load(f)

In [8]:
all_vec.shape

(6187432, 100)

In [30]:
vec_foa_df = pd.DataFrame(vec_foa)
vec_foa_df = vec_foa_df.set_index(pd.Index(foa_abstracts['Award_number'].values.tolist()))

In [31]:
grant_researcher = foa_abstracts[['Award_number', 'researcher_id']]

grant2re = grant_researcher.set_index('Award_number').T.to_dict('records')[0]

vec_df = pd.DataFrame(all_vec)

vec_df = vec_df.set_index(pd.Index(pubid['publication_id'].values.tolist()))

In [32]:
all_PI_idx = pd.read_csv('./data/all_PI_idx.csv')
sampled_id = all_PI_idx[all_PI_idx['category'] != 'potential']['researcher_id'].unique()

In [54]:
matching_dict = {}

matching_dict['funded_name'] = []
matching_dict['funded_id'] = []
matching_dict['funded_npubs'] = []
matching_dict['funded_entropy'] = []
matching_dict['funded_FOA_similarity'] = []
matching_dict['matched_name'] = []
matching_dict['matched_id'] = []
matching_dict['matched_cosine_similarity'] = []
matching_dict['matched_npubs'] = []
matching_dict['matched_entropy'] = []
matching_dict['matched_FOA_similarity'] = []



for target_rid in tqdm(sampled_id):
    target_name = id2name[target_rid]
    grant_years = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['StartYear'].values
    if len(grant_years) != 0:
        grant_year = grant_years[0]
        Award_number = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['Award_number'].values[0]
        Award_LDA = vec_foa_df.loc[Award_number].values

        tmp_pub = pub2re[(pub2re['researcher_id'] == target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]
        target_pubcount = len(tmp_pub)
        if target_pubcount >= 1:
            tmp_pub = tmp_pub.set_index(pd.Index(tmp_pub['publication_id'].values.tolist())).drop(['publication_id'], axis = 1)

            tmp_lda = vec_df.\
                join(tmp_pub, how = 'inner').drop(['researcher_id'], axis = 1)

            target_lda = tmp_lda.mean(axis = 0).values
            target_entropy = entropy(target_lda)

            tmp_other_pub = pub2re[(pub2re['researcher_id'] != target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]

            other_researcher_count = tmp_other_pub.groupby('researcher_id').count().reset_index()
            other_researcher_id = other_researcher_count[other_researcher_count['publication_id'] >= 10]['researcher_id'].values
            other_researcher_pubcount = other_researcher_count[other_researcher_count['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)
            other_researcher_pubcount = other_researcher_pubcount.set_index(pd.Index(other_researcher_pubcount['researcher_id'].values.tolist())).drop(['researcher_id'], axis = 1)

            tmp_other_pub = tmp_other_pub[tmp_other_pub['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)

            other_lda = vec_df.\
                reset_index().rename({'index': 'publication_id'}, axis = 1).\
                merge(tmp_other_pub, on = 'publication_id', how = 'inner').\
                drop(['publication_id'], axis = 1).groupby('researcher_id').mean()

            other_rid = other_lda.index.tolist()
            other_entropy = entropy(other_lda.values, axis = 1)
            top_idx = np.argsort(cosine_similarity(target_lda.reshape(1, -1), other_lda.values))[0][-50:][::-1]
            top_dist = cosine_similarity(target_lda.reshape(1, -1), other_lda.values)[0][top_idx]
            top_rid = [other_rid[idx] for idx in top_idx]
            top_name = [id2name[rid] for rid in top_rid]

            top_entropy = [other_entropy[idx] for idx in top_idx]
            top_pubcount = [other_researcher_pubcount.loc[rid]['publication_id'] for rid in top_rid]
            top_lda = other_lda.loc[top_rid]

            other_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), top_lda.values)

            target_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), target_lda.reshape(1, -1))
            
            matching_dict['funded_name'] += [target_name] * len(top_name)
            matching_dict['funded_id'] += [target_rid] * len(top_name)
            matching_dict['funded_npubs'] += [target_pubcount] * len(top_name)
            matching_dict['funded_entropy'] += [target_entropy] * len(top_name)
            matching_dict['funded_FOA_similarity'] += [target_foa_dist[0][0]] * len(top_name)
            matching_dict['matched_name'] += top_name
            matching_dict['matched_id'] += top_rid
            matching_dict['matched_cosine_similarity'] += top_dist.tolist()
            matching_dict['matched_npubs'] += top_pubcount
            matching_dict['matched_entropy'] += top_entropy
            matching_dict['matched_FOA_similarity'] += other_foa_dist[0].tolist()



In [55]:
pd.DataFrame(matching_dict)

,funded_name,funded_id,funded_npubs,funded_entropy,funded_FOA_similarity,matched_name,matched_id,matched_cosine_similarity,matched_npubs,matched_entropy,matched_FOA_similarity
0,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Emmanuel G Collins,ur.011471236145.91,1.0,54,4.605146,0.999971
1,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Joydeep Acharya,ur.010701015017.59,1.0,11,4.605147,0.999972
2,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Bren C Mochocki,ur.013765276155.27,1.0,11,4.605147,0.999975
3,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Lei Wu,ur.010322327104.06,1.0,12,4.605145,0.999969
4,Santiago C Grijalva,ur.013335016502.92,19,4.605146,0.999973,Yu Ding,ur.015716633211.16,1.0,46,4.605148,0.999973
...,...,...,...,...,...,...,...,...,...,...,...
5995,Zhi-Hua Qu,ur.016570352533.21,109,4.605146,0.999950,Sean A Ramprashad,ur.010162606610.47,1.0,37,4.605143,0.999945
5996,Zhi-Hua Qu,ur.016570352533.21,109,4.605146,0.999950,Joydeep Acharya,ur.010701015017.59,1.0,15,4.605142,0.999951
5997,Zhi-Hua Qu,ur.016570352533.21,109,4.605146,0.999950,Kia Bazargan,ur.013344732015.91,1.0,38,4.605150,0.999957
5998,Zhi-Hua Qu,ur.016570352533.21,109,4.605146,0.999950,Haomiao Huang,ur.012561141107.08,1.0,15,4.605145,0.999947


In [56]:
pd.DataFrame(matching_dict).\
    merge(potential_author_citing.\
              rename({'researcher_id': 'funded_id', 'citing_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    merge(potential_author_referenced.\
              rename({'researcher_id': 'funded_id', 'referenced_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    fillna(0).to_csv('./data/matching_foa_researcher_124_d100_66.csv', index = False)
    
    
    

In [ ]:
coauthorship_weight = pd.read_csv('./data/pubid_coauthor_count.csv')
coauthorship_weight = coauthorship_weight.set_index(coauthorship_weight['publication_id']).drop(['publication_id'], axis = 1)
weighted_vec = coauthorship_weight.join(vec_df)
weighted_vec = weighted_vec.iloc[:, :].div(weighted_vec.coauthor_count, axis=0)
weighted_vec = weighted_vec.drop('coauthor_count', axis = 1)
coauthorship_weight['weight'] = coauthorship_weight['coauthor_count'].apply(lambda x: 1 / x)
coauthorship_weight = pd.read_csv('./data/pubid_coauthor_count.csv')

In [38]:
matching_dict = {}

matching_dict['funded_name'] = []
matching_dict['funded_id'] = []
matching_dict['funded_npubs'] = []
matching_dict['funded_entropy'] = []
matching_dict['funded_FOA_similarity'] = []
matching_dict['matched_name'] = []
matching_dict['matched_id'] = []
matching_dict['matched_cosine_similarity'] = []
matching_dict['matched_npubs'] = []
matching_dict['matched_entropy'] = []
matching_dict['matched_FOA_similarity'] = []



for target_rid in tqdm(sampled_id):
    target_name = id2name[target_rid]
    grant_years = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['StartYear'].values
    if len(grant_years) != 0:
        grant_year = grant_years[0]
        Award_number = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['Award_number'].values[0]
        Award_LDA = vec_foa_df.loc[Award_number].values

        tmp_pub = pub2re[(pub2re['researcher_id'] == target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]
        target_pubcount = len(tmp_pub)
        if target_pubcount >= 1:
            tmp_pub = tmp_pub.set_index(pd.Index(tmp_pub['publication_id'].values.tolist())).drop(['publication_id'], axis = 1)

            tmp_lda = weighted_vec.\
                join(tmp_pub, how = 'inner').drop(['researcher_id'], axis = 1)

            target_lda = tmp_lda.mean(axis = 0).values
            target_entropy = entropy(target_lda)

            tmp_other_pub = pub2re[(pub2re['researcher_id'] != target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]

            other_researcher_count = tmp_other_pub.groupby('researcher_id').count().reset_index()
            other_researcher_id = other_researcher_count[other_researcher_count['publication_id'] >= 10]['researcher_id'].values
            other_researcher_pubcount = other_researcher_count[other_researcher_count['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)
            other_researcher_pubcount = other_researcher_pubcount.set_index(pd.Index(other_researcher_pubcount['researcher_id'].values.tolist())).drop(['researcher_id'], axis = 1)

            tmp_other_pub = tmp_other_pub[tmp_other_pub['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)

            other_lda = weighted_vec.\
                reset_index().rename({'index': 'publication_id'}, axis = 1).\
                merge(tmp_other_pub, on = 'publication_id', how = 'inner').\
                drop(['publication_id'], axis = 1).groupby('researcher_id').mean()

            other_rid = other_lda.index.tolist()
            other_entropy = entropy(other_lda.values, axis = 1)
            top_idx = np.argsort(cosine_similarity(target_lda.reshape(1, -1), other_lda.values))[0][-100:][::-1]
            top_dist = cosine_similarity(target_lda.reshape(1, -1), other_lda.values)[0][top_idx]
            top_rid = [other_rid[idx] for idx in top_idx]
            top_name = [id2name[rid] for rid in top_rid]

            top_entropy = [other_entropy[idx] for idx in top_idx]
            top_pubcount = [other_researcher_pubcount.loc[rid]['publication_id'] for rid in top_rid]
            top_lda = other_lda.loc[top_rid]

            other_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), top_lda.values)

            target_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), target_lda.reshape(1, -1))
            
            matching_dict['funded_name'] += [target_name] * len(top_name)
            matching_dict['funded_id'] += [target_rid] * len(top_name)
            matching_dict['funded_npubs'] += [target_pubcount] * len(top_name)
            matching_dict['funded_entropy'] += [target_entropy] * len(top_name)
            matching_dict['funded_FOA_similarity'] += [target_foa_dist[0][0]] * len(top_name)
            matching_dict['matched_name'] += top_name
            matching_dict['matched_id'] += top_rid
            matching_dict['matched_cosine_similarity'] += top_dist.tolist()
            matching_dict['matched_npubs'] += top_pubcount
            matching_dict['matched_entropy'] += top_entropy
            matching_dict['matched_FOA_similarity'] += other_foa_dist[0].tolist()

In [40]:
pd.DataFrame(matching_dict)

,funded_name,funded_id,funded_npubs,funded_entropy,funded_FOA_similarity,matched_name,matched_id,matched_cosine_similarity,matched_npubs,matched_entropy,matched_FOA_similarity
0,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Jorge F Valenzuela,ur.012106654411.32,1.000000,22,4.605148,0.999975
1,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Chunting Mi,ur.011434721140.49,1.000000,13,4.605147,0.999975
2,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Daji Qiao,ur.07611240351.64,1.000000,52,4.605147,0.999974
3,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Girish Vinayak Chowdhary,ur.016663530513.65,1.000000,13,4.605146,0.999977
4,Santiago C Grijalva,ur.013335016502.92,19,4.605147,0.999975,Ilya V Kolmanovsky,ur.016131577413.61,1.000000,143,4.605149,0.999973
...,...,...,...,...,...,...,...,...,...,...,...
11995,Zhi-Hua Qu,ur.016570352533.21,109,4.605145,0.999944,Jack D Reilly,ur.012423357433.35,0.999999,12,4.605139,0.999937
11996,Zhi-Hua Qu,ur.016570352533.21,109,4.605145,0.999944,John J Hasenbein,ur.013040047235.99,0.999999,21,4.605147,0.999941
11997,Zhi-Hua Qu,ur.016570352533.21,109,4.605145,0.999944,Leonardo Bobadilla,ur.013073650035.26,0.999999,11,4.605143,0.999938
11998,Zhi-Hua Qu,ur.016570352533.21,109,4.605145,0.999944,Nathan Wayne Fisher,ur.013125433747.65,0.999999,50,4.605142,0.999937


In [53]:
pd.DataFrame(matching_dict).\
    merge(potential_author_citing.\
              rename({'researcher_id': 'funded_id', 'citing_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    merge(potential_author_referenced.\
              rename({'researcher_id': 'funded_id', 'referenced_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    fillna(0).to_csv('./data/matching_foa_researcher_124_d100_66_weighted.csv', index = False)
    
    
    

In [ ]:
j = 0
all_vec = np.load(open('./data/lda/d100/embedding_default/d100_lda_part' + str(j) + '.npy', 'rb'))
for j in tqdm(range(1, 62)):
    all_vec = np.vstack([all_vec, np.load(open('./data/lda/d100/embedding_default/d100_lda_part' + str(j) + '.npy', 'rb'))])

In [ ]:
matching_dict = {}

matching_dict['funded_name'] = []
matching_dict['funded_id'] = []
matching_dict['funded_npubs'] = []
matching_dict['funded_entropy'] = []
matching_dict['funded_FOA_similarity'] = []
matching_dict['matched_name'] = []
matching_dict['matched_id'] = []
matching_dict['matched_cosine_similarity'] = []
matching_dict['matched_npubs'] = []
matching_dict['matched_entropy'] = []
matching_dict['matched_FOA_similarity'] = []



for target_rid in tqdm(sampled_id):
    target_name = id2name[target_rid]
    grant_years = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['StartYear'].values
    if len(grant_years) != 0:
        grant_year = grant_years[0]
        Award_number = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['Award_number'].values[0]
        Award_LDA = vec_foa_df.loc[Award_number].values

        tmp_pub = pub2re[(pub2re['researcher_id'] == target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]
        target_pubcount = len(tmp_pub)
        if target_pubcount >= 1:
            tmp_pub = tmp_pub.set_index(pd.Index(tmp_pub['publication_id'].values.tolist())).drop(['publication_id'], axis = 1)

            tmp_lda = vec_df.\
                join(tmp_pub, how = 'inner').drop(['researcher_id'], axis = 1)

            target_lda = tmp_lda.mean(axis = 0).values
            target_entropy = entropy(target_lda)

            tmp_other_pub = pub2re[(pub2re['researcher_id'] != target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]

            other_researcher_count = tmp_other_pub.groupby('researcher_id').count().reset_index()
            other_researcher_id = other_researcher_count[other_researcher_count['publication_id'] >= 10]['researcher_id'].values
            other_researcher_pubcount = other_researcher_count[other_researcher_count['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)
            other_researcher_pubcount = other_researcher_pubcount.set_index(pd.Index(other_researcher_pubcount['researcher_id'].values.tolist())).drop(['researcher_id'], axis = 1)

            tmp_other_pub = tmp_other_pub[tmp_other_pub['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)

            other_lda = vec_df.\
                reset_index().rename({'index': 'publication_id'}, axis = 1).\
                merge(tmp_other_pub, on = 'publication_id', how = 'inner').\
                drop(['publication_id'], axis = 1).groupby('researcher_id').mean()

            other_rid = other_lda.index.tolist()
            other_entropy = entropy(other_lda.values, axis = 1)
            top_idx = np.argsort(cosine_similarity(target_lda.reshape(1, -1), other_lda.values))[0][-50:][::-1]
            top_dist = cosine_similarity(target_lda.reshape(1, -1), other_lda.values)[0][top_idx]
            top_rid = [other_rid[idx] for idx in top_idx]
            top_name = [id2name[rid] for rid in top_rid]

            top_entropy = [other_entropy[idx] for idx in top_idx]
            top_pubcount = [other_researcher_pubcount.loc[rid]['publication_id'] for rid in top_rid]
            top_lda = other_lda.loc[top_rid]

            other_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), top_lda.values)

            target_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), target_lda.reshape(1, -1))
            
            matching_dict['funded_name'] += [target_name] * len(top_name)
            matching_dict['funded_id'] += [target_rid] * len(top_name)
            matching_dict['funded_npubs'] += [target_pubcount] * len(top_name)
            matching_dict['funded_entropy'] += [target_entropy] * len(top_name)
            matching_dict['funded_FOA_similarity'] += [target_foa_dist[0][0]] * len(top_name)
            matching_dict['matched_name'] += top_name
            matching_dict['matched_id'] += top_rid
            matching_dict['matched_cosine_similarity'] += top_dist.tolist()
            matching_dict['matched_npubs'] += top_pubcount
            matching_dict['matched_entropy'] += top_entropy
            matching_dict['matched_FOA_similarity'] += other_foa_dist[0].tolist()



In [ ]:
pd.DataFrame(matching_dict)

In [ ]:
pd.DataFrame(matching_dict).\
    merge(potential_author_citing.\
              rename({'researcher_id': 'funded_id', 'citing_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    merge(potential_author_referenced.\
              rename({'researcher_id': 'funded_id', 'referenced_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    fillna(0).to_csv('./data/matching_foa_researcher_124_d100_default.csv', index = False)

In [ ]:
matching_dict = {}

matching_dict['funded_name'] = []
matching_dict['funded_id'] = []
matching_dict['funded_npubs'] = []
matching_dict['funded_entropy'] = []
matching_dict['funded_FOA_similarity'] = []
matching_dict['matched_name'] = []
matching_dict['matched_id'] = []
matching_dict['matched_cosine_similarity'] = []
matching_dict['matched_npubs'] = []
matching_dict['matched_entropy'] = []
matching_dict['matched_FOA_similarity'] = []



for target_rid in tqdm(sampled_id):
    target_name = id2name[target_rid]
    grant_years = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['StartYear'].values
    if len(grant_years) != 0:
        grant_year = grant_years[0]
        Award_number = foa_abstracts[foa_abstracts['researcher_id'] == target_rid]['Award_number'].values[0]
        Award_LDA = vec_foa_df.loc[Award_number].values

        tmp_pub = pub2re[(pub2re['researcher_id'] == target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]
        target_pubcount = len(tmp_pub)
        if target_pubcount >= 1:
            tmp_pub = tmp_pub.set_index(pd.Index(tmp_pub['publication_id'].values.tolist())).drop(['publication_id'], axis = 1)

            tmp_lda = weighted_vec.\
                join(tmp_pub, how = 'inner').drop(['researcher_id'], axis = 1)

            target_lda = tmp_lda.mean(axis = 0).values
            target_entropy = entropy(target_lda)

            tmp_other_pub = pub2re[(pub2re['researcher_id'] != target_rid) & (pub2re['publication_id'].isin(pubid_year[(pubid_year['year'] < grant_year) & (pubid_year['year'] >= (grant_year - 10))]['publication_id'].values))]

            other_researcher_count = tmp_other_pub.groupby('researcher_id').count().reset_index()
            other_researcher_id = other_researcher_count[other_researcher_count['publication_id'] >= 10]['researcher_id'].values
            other_researcher_pubcount = other_researcher_count[other_researcher_count['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)
            other_researcher_pubcount = other_researcher_pubcount.set_index(pd.Index(other_researcher_pubcount['researcher_id'].values.tolist())).drop(['researcher_id'], axis = 1)

            tmp_other_pub = tmp_other_pub[tmp_other_pub['researcher_id'].isin(other_researcher_id)].reset_index(drop = True)

            other_lda = weighted_vec.\
                reset_index().rename({'index': 'publication_id'}, axis = 1).\
                merge(tmp_other_pub, on = 'publication_id', how = 'inner').\
                drop(['publication_id'], axis = 1).groupby('researcher_id').mean()

            other_rid = other_lda.index.tolist()
            other_entropy = entropy(other_lda.values, axis = 1)
            top_idx = np.argsort(cosine_similarity(target_lda.reshape(1, -1), other_lda.values))[0][-100:][::-1]
            top_dist = cosine_similarity(target_lda.reshape(1, -1), other_lda.values)[0][top_idx]
            top_rid = [other_rid[idx] for idx in top_idx]
            top_name = [id2name[rid] for rid in top_rid]

            top_entropy = [other_entropy[idx] for idx in top_idx]
            top_pubcount = [other_researcher_pubcount.loc[rid]['publication_id'] for rid in top_rid]
            top_lda = other_lda.loc[top_rid]

            other_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), top_lda.values)

            target_foa_dist = cosine_similarity(Award_LDA.reshape(1, -1), target_lda.reshape(1, -1))
            
            matching_dict['funded_name'] += [target_name] * len(top_name)
            matching_dict['funded_id'] += [target_rid] * len(top_name)
            matching_dict['funded_npubs'] += [target_pubcount] * len(top_name)
            matching_dict['funded_entropy'] += [target_entropy] * len(top_name)
            matching_dict['funded_FOA_similarity'] += [target_foa_dist[0][0]] * len(top_name)
            matching_dict['matched_name'] += top_name
            matching_dict['matched_id'] += top_rid
            matching_dict['matched_cosine_similarity'] += top_dist.tolist()
            matching_dict['matched_npubs'] += top_pubcount
            matching_dict['matched_entropy'] += top_entropy
            matching_dict['matched_FOA_similarity'] += other_foa_dist[0].tolist()

In [ ]:
pd.DataFrame(matching_dict)

In [ ]:
pd.DataFrame(matching_dict).\
    merge(potential_author_citing.\
              rename({'researcher_id': 'funded_id', 'citing_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    merge(potential_author_referenced.\
              rename({'researcher_id': 'funded_id', 'referenced_researcher_id': 'matched_id'}, axis = 1),
          on = ['funded_id', 'matched_id'], 
          how = 'left').\
    fillna(0).to_csv('./data/matching_foa_researcher_124_d100_default_weighted.csv', index = False)
    
    
    